<a href="https://colab.research.google.com/github/dakotalock/QuantumBlue/blob/main/%5BMoon_Shot%5D_Quantum_Blue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras.callbacks
!pip install Dill
!pip install chess
!pip install numpy
!pip install qiskit
!pip install qiskit_machine_learning
!pip install qiskit-aer
!pip install progressbar
!pip install tqdm
!gdown https://drive.google.com/file/d/1H5tOegKJutjGVhzYqChXw0Iq9o4jWDVa/view?usp=drive_link -O Carlsen.pgn
!gdown https://drive.google.com/file/d/1-7oN8BOUmxhpWrhfIh6tv5ATknFoWrwa/view?usp=drive_link -O vqc_model.dill

In [6]:
#@title  Original code `quantum_blue_prime.py`

import dill

import os

import chess

import numpy as np

import random

from tqdm import tqdm

from qiskit.circuit.library import EfficientSU2

from qiskit.algorithms.optimizers import COBYLA

from qiskit_machine_learning.algorithms import VQC

from qiskit_machine_learning.neural_networks import TwoLayerQNN

from qiskit import Aer

from qiskit.utils import QuantumInstance

from qiskit.circuit import ParameterVector

backend = Aer.get_backend("qasm_simulator")

quantum_instance = QuantumInstance(
    backend, shots=1024, seed_simulator=42, seed_transpiler=42
)


import tensorflow as tf

print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
except ValueError:
    raise BaseException(
        "ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!"
    )
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)


def material_score(board):
    material_weights = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 0,
    }

    score = 0

    for piece_type in material_weights.keys():
        score += (
            len(board.pieces(piece_type, chess.WHITE)) * material_weights[piece_type]
        )

        score -= (
            len(board.pieces(piece_type, chess.BLACK)) * material_weights[piece_type]
        )
    return score


def mobility_score(board):
    mobility_weight = 10

    score = 0

    score += len(list(board.legal_moves)) * mobility_weight

    return score


def tempo_score(board):
    tempo_weight = 10

    score = 0

    if board.turn == chess.WHITE:
        score += tempo_weight
    else:
        score -= tempo_weight
    return score


def pawn_structure_score(board):
    doubled_pawn_weight = -20

    score = 0

    for file in range(8):
        white_pawns = board.pieces(chess.PAWN, chess.WHITE) & chess.BB_FILES[file]

        black_pawns = board.pieces(chess.PAWN, chess.BLACK) & chess.BB_FILES[file]

        if white_pawns:
            score += doubled_pawn_weight * (bin(white_pawns).count("1") - 1)
        if black_pawns:
            score -= doubled_pawn_weight * (bin(black_pawns).count("1") - 1)
    return score


def king_safety_score(board):
    king_safety_weight = 20

    score = 0

    white_king_square = board.king(chess.WHITE)

    black_king_square = board.king(chess.BLACK)

    if white_king_square is not None:
        score += king_safety_weight * (7 - chess.square_rank(white_king_square))
    if black_king_square is not None:
        score -= king_safety_weight * chess.square_rank(black_king_square)
    return score


def Qfish_Evaluation(board):
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0,
    }

    score = 0

    for piece_type in piece_values.keys():
        for square in board.pieces(piece_type, chess.WHITE):
            piece = board.piece_at(square)

            score += piece_values[piece_type]
        for square in board.pieces(piece_type, chess.BLACK):
            piece = board.piece_at(square)

            score -= piece_values[piece_type]
    # Material score

    score += material_score(board)

    # Mobility score

    score += mobility_score(board)

    # Tempo score

    score += tempo_score(board)

    # Pawn structure score

    score += pawn_structure_score(board)

    # King safety score

    score += king_safety_score(board)

    return score


def get_best_move(board):
    depth = 3
    alpha = -float("inf")
    beta = float("inf")
    best_move = None
    for move in list(board.legal_moves):
        new_board = board.copy()
        new_board.push(move)
        score = alpha_beta(new_board, depth, alpha, beta, True)
        if score > alpha:
            alpha = score
            best_move = move
    return best_move


def alpha_beta(board, depth, alpha, beta, maximizing_player):
    if depth == 0:
        return material_score(board) + Qfish_Evaluation(board) + VQC_Evaluation(board)
    if maximizing_player:
        best_score = -float("inf")
        for move in list(board.legal_moves):
            new_board = board.copy()
            new_board.push(move)
            score = (
                alpha_beta(new_board, depth - 1, alpha, beta, False)
                + Qfish_Evaluation(new_board)
                + VQC_Evaluation(new_board)
            )
            best_score = max(best_score, score)
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break
        return best_score
    else:
        best_score = float("inf")
        for move in list(board.legal_moves):
            new_board = board.copy()
            new_board.push(move)
            score = (
                alpha_beta(new_board, depth - 1, alpha, beta, True)
                + Qfish_Evaluation(new_board)
                + VQC_Evaluation(new_board)
            )
            best_score = min(best_score, score)
            beta = min(beta, best_score)
            if beta <= alpha:
                break
        return best_score
    return best_move


def limit_to_legal_moves(predicted_moves, board):
    legal_moves = list(board.legal_moves)

    legal_move_indices = [
        i for i, move in enumerate(predicted_moves) if move in legal_moves
    ]

    if not legal_move_indices:
        return random.choice(legal_moves)
    best_move_index = legal_move_indices[np.argmax(predicted_moves[legal_move_indices])]

    return predicted_moves[best_move_index]


# Create a classical feature extraction function
def extract_features(board):
    features = []

    for sq in chess.SQUARES:
        piece = board.piece_at(sq)
        if piece:
            piece_type = piece.piece_type
            features.append(piece_type)
        else:
            features.append(0)
    return np.array(features)


# Define the quantum feature map and create a QuantumInstance
num_qubits = 64
quantum_feature_map = EfficientSU2(num_qubits)
quantum_ansatz = EfficientSU2(num_qubits)

from qiskit.circuit import Parameter, QuantumCircuit


def create_custom_circuit(num_qubits, prefix):
    custom_circuit = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        custom_circuit.ry(Parameter(f"{prefix}theta[{i}]"), i)
    return custom_circuit


# Create separate circuits for the quantum_feature_map and the quantum_ansatz
quantum_feature_map = create_custom_circuit(num_qubits, prefix="fm_")
quantum_ansatz = create_custom_circuit(num_qubits, prefix="a_")

# Change the optimizer to SPSA
from qiskit.algorithms.optimizers import SPSA

classical_optimizer = SPSA()

# Load the saved VQC model from the dill file

with open(r"/content/vqc_model.dill", "rb") as f:
    loaded_vqc = dill.load(f)

    # Import additional libraries
import chess.pgn

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error


# Read the PGN file


def read_pgn_file(file_path):
    with open(file_path) as pgn_file:
        games = []

        while True:
            game = chess.pgn.read_game(pgn_file)

            if game is None:
                break
            games.append(game)
    return games


# Preprocess the games


def preprocess_games(games):
    features = []

    labels = []

    for game in games:
        board = game.board()

        for move in game.mainline_moves():
            board.push(move)

            features.append(extract_features(board))

            labels.append(Qfish_Evaluation(board))
    return np.array(features), np.array(labels)


# Load the games from the PGN file

print("Reading PGN file...")

games = read_pgn_file(r"/content/Carlsen.pgn")

print("PGN file read.")


# Preprocess the games to extract features and labels

print("Preprocessing games...")

X, y = preprocess_games(games)

print("Games preprocessed.")

# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


epochs = 1000  # Set number of epochs


print("Training started!")

# Create an autosave callback

from keras.callbacks import ModelCheckpoint

filepath = "vqc_model.ckpt"

autosave_callback = ModelCheckpoint(
    filepath, monitor="val_loss", verbose=1, save_best_only=True
)


def train_model(epochs):
    for epoch in tqdm(range(epochs), desc="Training Progress"):
        loaded_vqc.fit(X_train, y_train)


print("Training complete!")


# Save trained model
def save_model(model):
    file_path = "QuantumBlue.dill"
    try:
        with open(file_path, "wb") as f:
            dill.dump(model, f)
        print(f"Dill file {file_path} saved successfully!")
    except IOError:
        print(f"Unable to save dill file to {file_path}.")


save_model(loaded_vqc)


def get_ai_move(board):
    depth = 3
    alpha = -float("inf")
    beta = float("inf")
    best_move = None
    for move in list(board.legal_moves):
        new_board = board.copy()
        new_board.push(move)
        score = (
            alpha_beta(new_board, depth, alpha, beta, True)
            + Qfish_Evaluation(new_board)
            + VQC_Evaluation(new_board)
        )
        if score > alpha:
            alpha = score
            best_move = move
    return best_move


def get_user_move(board):
    legal_moves = list(board.legal_moves)

    print("Legal moves:", [str(move) for move in legal_moves])

    user_move = input("Enter your move: ")

    while chess.Move.from_uci(user_move) not in legal_moves:
        print("Invalid move. Please enter a legal move.")

        user_move = input("Enter your move: ")
    return chess.Move.from_uci(user_move)


board = chess.Board()


while not board.is_game_over():
    print(board)

    if board.turn:  # White's turn (user)
        user_move = get_user_move(board)

        board.push(user_move)
    else:  # Black's turn (AI)
        ai_move = get_ai_move(board)

        board.push(ai_move)

        print("AI move:", ai_move)
print("Game over. Result:", board.result())

<ipython-input-6-c87d7517e269>:31: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(


Tensorflow version 2.12.0
Running on TPU  ['10.124.48.98:8470']


UnpicklingError: ignored

In [ ]:
#@title QuantumChessAI

class QuantumChessAI:
    """
    QuantumChessAI is a class for training, evaluating, and making predictions with a Variational Quantum Classifier (VQC) model.

    Attributes:
        model: An instance of Qiskit's VQC model or a loaded model from a file.

    Methods:
        __init__(self, model_path=None): Constructor that initializes a new VQC model or loads an existing one from a file.
        train(self, X_train, y_train, epochs=1000, checkpoint_interval=1000): Trains the VQC model on provided data.
        evaluate(self, X_test, y_test): Evaluates the model's performance on testing data.
        predict(self, X): Uses the trained model to make predictions on provided data.
        save(self, model_path): Saves the current model to a file.

    Example usage:
        ai = QuantumChessAI()
        ai.train(X_train, y_train)
        score = ai.evaluate(X_test, y_test)
        print("Test score:", score)
        pred = ai.predict(X)
        ai.save('QuantumChessAI.dill')
    """
    import dill
    from qiskit_machine_learning.algorithms import VQC
    from qiskit.circuit.library import EfficientSU2
    from qiskit.algorithms.optimizers import SPSA
    from qiskit.utils import QuantumInstance
    from qiskit import Aer
    from keras.callbacks import ModelCheckpoint
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error

    def __init__(self, model_path=None):
        num_qubits = 64
        quantum_feature_map = self.EfficientSU2(num_qubits)
        quantum_ansatz = self.EfficientSU2(num_qubits)
        backend = self.Aer.get_backend('qasm_simulator')
        quantum_instance = self.QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)
        classical_optimizer = self.SPSA()
        self.model = self.VQC(feature_map=quantum_feature_map,
                              ansatz=quantum_ansatz,
                              optimizer=classical_optimizer,
                              quantum_instance=quantum_instance)

        # If a model_path is provided, load the model from the file
        if model_path is not None:
            with open(model_path, 'rb') as f:
                self.model = self.dill.load(f)

    def train(self, X_train, y_train, epochs=1000, checkpoint_interval=1000):
        filepath = "vqc_model_checkpoint.h5"
        checkpoint_callback = self.ModelCheckpoint(filepath, monitor="loss", verbose=1,
                                                   save_best_only=True, save_freq=checkpoint_interval)
        for epoch in range(epochs):
            self.model.fit(X_train, y_train, callbacks=[checkpoint_callback])
        return self.model

    def evaluate(self, X_test, y_test):
        return self.model.score(X_test, y_test)

    def predict(self, X):
        return self.model.predict(X)

    def save(self, model_path):
        try:
            with open(model_path, 'wb') as f:
                self.dill.dump(self.model, f)
            print(f"Dill file {model_path} saved successfully!")
        except IOError:
            print(f"Unable to save dill file to {model_path}.")


In [ ]:
#@title Use Case

# Initialization
ai = QuantumChessAI()

# Training
ai.train(X_train, y_train)

# Saving the model
ai.save('QuantumChessAI.dill')

# Evaluation
score = ai.evaluate(X_test, y_test)
print("Test score:", score)

# Predicting
pred = ai.predict(X)
